<a href="https://colab.research.google.com/github/andre31072003/Projeto_Final/blob/main/Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.8/782.8 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Instalar o YOLOv8

In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo mode=checks

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 532, in entrypoint
    raise ValueError(f"Invalid 'mode={mode}'. Valid modes are {MODES}.\n{CLI_HELP_MSG}")
ValueError: Invalid 'mode=<module 'ultralytics.utils.checks' from '/usr/local/lib/python3.10/dist-packages/ultralytics/utils/checks.py'>'. Valid modes are {'val', 'track', 'train', 'export', 'benchmark', 'predict'}.

    Arguments received: ['yolo', 'mode=checks']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'segment', 'pose', 'classify', 'obb', 'detect'}
                MODE (required) is one of {'val', 'track', 'train', 'export', 'benchmark', 'predict'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at ht

Adicionar video e analisar video
Dividido apenas em 2 setores


In [ ]:
from google.colab import files
from google.colab.patches import cv2_imshow
import cv2
from ultralytics import YOLO
import os
import pandas as pd
import time

# Fazer upload do vídeo
uploaded = files.upload()

# Verificar se o arquivo foi carregado com sucesso
for filename in uploaded.keys():
    print(f'Arquivo {filename} carregado com sucesso.')

# Usar o nome do primeiro arquivo carregado
video_path = next(iter(uploaded.keys()))

# Lista de modelos YOLOv8 a serem utilizados
model_files = ['yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt'] # depois voltar a adicionar 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt'
results_list = []

# Função para verificar se a classe é um veículo
def is_vehicle(cls_id):
    vehicle_classes = [2, 3, 5, 7]  # COCO dataset IDs for car, motorcycle, bus, truck
    return cls_id in vehicle_classes

for model_file in model_files:
    print(f'Analisando com o modelo {model_file}...')
    # Carregar o modelo YOLOv8 pré-treinado
    model = YOLO(model_file)

    cap = cv2.VideoCapture(video_path)

    # Verificar se o vídeo foi aberto com sucesso
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        continue

    # Obter informações do vídeo
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Configurar o writer para salvar o vídeo
    output_path = f'output_video_{model_file}.mp4'
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Definir o intervalo de análise de frames
    frame_interval = 5
    frame_count = 0

    # Dicionário para armazenar as posições anteriores dos veículos
    previous_positions = {}

    # Variáveis para contar ultrapassagens
    ultrapassagens_direita = 0
    ultrapassagens_esquerda = 0

    # Variável para medir o tempo de processamento
    total_detection_time = 0
    detection_count = 0

    # Loop para ler e processar cada frame do vídeo
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Verificar se é hora de processar o frame atual
        if frame_count % frame_interval == 0:
            # Aplicar Canny Edge Detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 50, 150)

            # Definir as regiões de interesse (ROI) para a estrada direita e esquerda
            height, width = frame.shape[:2]
            roi_left = (0, height//2, width//2, height)
            roi_right = (width//2, height//2, width, height)

            start_time = time.time()
            # Fazer a detecção e rastreamento
            results = model.track(frame, persist=True, stream=True)
            detection_time = time.time() - start_time
            total_detection_time += detection_time
            detection_count += 1

            # Iterar sobre as detecções e desenhar as caixas delimitadoras
            current_positions = {}
            for result in results:
                for box in result.boxes:
                    if is_vehicle(int(box.cls[0])):  # Verificar se a classe é um veículo
                        x1, y1, x2, y2 = box.xyxy[0].tolist()  # Convertendo para lista de inteiros
                        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                        vehicle_id = box.id[0].item()  # Obter o ID do veículo

                        current_positions[vehicle_id] = (x1, y1, x2, y2)

                        # Verificar se temos a posição anterior deste veículo
                        if vehicle_id in previous_positions:
                            prev_x1, prev_y1, prev_x2, prev_y2 = previous_positions[vehicle_id]

                            # Verificar se o veículo entra nas ROIs
                            is_in_left_roi = (y1 >= height//2 and y1 <= height and x1 <= width//2)
                            is_in_right_roi = (y1 >= height//2 and y1 <= height and x1 >= width//2)

                            if is_in_left_roi or is_in_right_roi:
                                # Calcular o deslocamento horizontal
                                displacement = x1 - prev_x1

                                # Determinar se o veículo está ultrapassando pela esquerda ou direita (invertido para câmera traseira)
                                if displacement > 0 and is_in_left_roi:
                                    direction = "Direita"
                                    ultrapassagens_direita += 1
                                elif displacement < 0 and is_in_right_roi:
                                    direction = "Esquerda"
                                    ultrapassagens_esquerda += 1
                                else:
                                    direction = None

                                if direction:
                                    # Adicionar texto indicando a direção da ultrapassagem
                                    cv2.putText(frame, f'Ultrapassando pela {direction}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                        # Atualizar a posição anterior do veículo
                        previous_positions[vehicle_id] = (x1, y1, x2, y2)

                        # Desenhar a caixa delimitadora
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Desenhar as linhas das ROIs na imagem
            cv2.line(frame, (0, height//2), (width, height//2), (255, 0, 0), 2)
            cv2.line(frame, (width//2, height//2), (width//2, height), (255, 0, 0), 2)

            # Escrever o frame processado no arquivo de saída
            out.write(frame)

        # Adicionar um delay para simular visualização em tempo real (ajuste conforme necessário)
        cv2.waitKey(int(1000 / fps))

    # Liberar o vídeo
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Armazenar os resultados
    average_detection_time = total_detection_time / detection_count if detection_count > 0 else 0
    results_list.append({
        'Modelo': model_file,
        'Ultrapassagens pela Direita': ultrapassagens_direita,
        'Ultrapassagens pela Esquerda': ultrapassagens_esquerda,
        'Tempo Médio de Detecção (s)': average_detection_time
    })

# Criar um DataFrame com os resultados
df_results = pd.DataFrame(results_list)

# Salvar o DataFrame em um arquivo Excel
excel_path = 'Comparacao_Modelos_YOLO.xlsx'
df_results.to_excel(excel_path, index=False)

# Fazer download do arquivo Excel
files.download(excel_path)

# Fazer download do vídeo processado
for model_file in model_files:
    files.download(f'output_video_{model_file}.mp4')


KeyboardInterrupt: 

Adicionar video e analisar video
Dividido apenas em 3 setores


In [ ]:
from google.colab import files
from google.colab.patches import cv2_imshow
import cv2
from ultralytics import YOLO
import os
import pandas as pd
import time

# Fazer upload do vídeo
uploaded = files.upload()

# Verificar se o arquivo foi carregado com sucesso
for filename in uploaded.keys():
    print(f'Arquivo {filename} carregado com sucesso.')

# Usar o nome do primeiro arquivo carregado
video_path = next(iter(uploaded.keys()))

# Lista de modelos YOLOv8 a serem utilizados
model_files = ['yolov8n.pt']  # depois voltar a adicionar 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt'
results_list = []

# Função para verificar se a classe é um veículo
def is_vehicle(cls_id):
    vehicle_classes = [2, 3, 5, 7]  # COCO dataset IDs for car, motorcycle, bus, truck
    return cls_id in vehicle_classes

for model_file in model_files:
    print(f'Analisando com o modelo {model_file}...')
    # Carregar o modelo YOLOv8 pré-treinado
    model = YOLO(model_file)

    cap = cv2.VideoCapture(video_path)

    # Verificar se o vídeo foi aberto com sucesso
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        continue

    # Obter informações do vídeo
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Configurar o writer para salvar o vídeo
    output_path = f'output_video_{model_file}.mp4'
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Definir o intervalo de análise de frames
    frame_interval = 5
    frame_count = 0

    # Dicionário para armazenar as posições anteriores dos veículos
    previous_positions = {}

    # Variáveis para contar ultrapassagens
    ultrapassagens_direita = 0
    ultrapassagens_esquerda = 0

    # Variável para medir o tempo de processamento
    total_detection_time = 0
    detection_count = 0

    # Loop para ler e processar cada frame do vídeo
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Verificar se é hora de processar o frame atual
        if frame_count % frame_interval == 0:
            # Aplicar Canny Edge Detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 50, 150)

            # Definir as regiões de interesse (ROI) para esquerda, central e direita
            height, width = frame.shape[:2]
            roi_left = (0, height//2, width//3, height)
            roi_center = (width//3, height//2, 2*width//3, height)
            roi_right = (2*width//3, height//2, width, height)

            start_time = time.time()
            # Fazer a detecção e rastreamento
            results = model.track(frame, persist=True, stream=True)
            detection_time = time.time() - start_time
            total_detection_time += detection_time
            detection_count += 1

            # Iterar sobre as detecções e desenhar as caixas delimitadoras
            current_positions = {}
            for result in results:
                for box in result.boxes:
                    if is_vehicle(int(box.cls[0])):  # Verificar se a classe é um veículo
                        x1, y1, x2, y2 = box.xyxy[0].tolist()  # Convertendo para lista de inteiros
                        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                        vehicle_id = box.id[0].item()  # Obter o ID do veículo

                        current_positions[vehicle_id] = (x1, y1, x2, y2)

                        # Verificar se temos a posição anterior deste veículo
                        if vehicle_id in previous_positions:
                            prev_x1, prev_y1, prev_x2, prev_y2 = previous_positions[vehicle_id]

                            # Verificar se o veículo entra nas ROIs
                            is_in_left_roi = (y1 >= height//2 and y1 <= height and x1 <= width//3)
                            is_in_right_roi = (y1 >= height//2 and y1 <= height and x1 >= 2*width//3)

                            if is_in_left_roi or is_in_right_roi:
                                # Calcular o deslocamento horizontal
                                displacement = x1 - prev_x1

                                # Determinar se o veículo está ultrapassando pela esquerda ou direita (invertido para câmera traseira)
                                if displacement > 0 and is_in_left_roi:
                                    direction = "Direita"
                                    ultrapassagens_direita += 1
                                elif displacement < 0 and is_in_right_roi:
                                    direction = "Esquerda"
                                    ultrapassagens_esquerda += 1
                                else:
                                    direction = None

                                if direction:
                                    # Adicionar texto indicando a direção da ultrapassagem
                                    cv2.putText(frame, f'Ultrapassando pela {direction}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                        # Atualizar a posição anterior do veículo
                        previous_positions[vehicle_id] = (x1, y1, x2, y2)

                        # Desenhar a caixa delimitadora
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Desenhar as linhas das ROIs na imagem
            cv2.line(frame, (0, height//2), (width, height//2), (255, 0, 0), 2)
            cv2.line(frame, (width//3, height//2), (width//3, height), (255, 0, 0), 2)
            cv2.line(frame, (2*width//3, height//2), (2*width//3, height), (255, 0, 0), 2)

            # Escrever o frame processado no arquivo de saída
            out.write(frame)

        # Adicionar um delay para simular visualização em tempo real (ajuste conforme necessário)
        cv2.waitKey(int(1000 / fps))

    # Liberar o vídeo
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Armazenar os resultados
    average_detection_time = total_detection_time / detection_count if detection_count > 0 else 0
    results_list.append({
        'Modelo': model_file,
        'Ultrapassagens pela Direita': ultrapassagens_direita,
        'Ultrapassagens pela Esquerda': ultrapassagens_esquerda,
        'Tempo Médio de Detecção (s)': average_detection_time
    })

# Criar um DataFrame com os resultados
df_results = pd.DataFrame(results_list)

# Salvar o DataFrame em um arquivo Excel
excel_path = 'Comparacao_Modelos_YOLO.xlsx'
df_results.to_excel(excel_path, index=False)

# Fazer download do arquivo Excel
files.download(excel_path)

# Fazer download do vídeo processado
for model_file in model_files:
    files.download(f'output_video_{model_file}.mp4')


Adicionar video e analisar video
Dividido com linhas obliquas

In [ ]:
from google.colab import files
import cv2
from ultralytics import YOLO
import os
import pandas as pd
import time
import psutil
import platform

# Fazer upload do vídeo
uploaded = files.upload()

# Verificar se o arquivo foi carregado com sucesso
for filename in uploaded.keys():
    print(f'Arquivo {filename} carregado com sucesso.')

# Usar o nome do primeiro arquivo carregado
video_path = next(iter(uploaded.keys()))

# Lista de modelos YOLOv8 a serem utilizados
model_files = ['yolov8m.pt', 'yolov8s.pt', 'yolov8n.pt', 'yolov8l.pt','yolov8x.pt' ]  # depois voltar a adicionar 'yolov8s.pt', 'yolov8n.pt', 'yolov8l.pt'
results_list = []

# Função para verificar se a classe é um veículo
def is_vehicle(cls_id):
    vehicle_classes = [2, 3, 5, 7]  # COCO dataset IDs for car, motorcycle, bus, truck
    return cls_id in vehicle_classes

for model_file in model_files:
    print(f'Analisando com o modelo {model_file}...')
    # Carregar o modelo YOLOv8 pré-treinado
    model = YOLO(model_file)

    cap = cv2.VideoCapture(video_path)

    # Verificar se o vídeo foi aberto com sucesso
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        continue

    # Obter informações do vídeo
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Configurar o writer para salvar o vídeo
    output_path = f'output_video_{model_file}.mp4'
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Definir o intervalo de análise de frames
    frame_interval = 5
    frame_count = 0

    # Dicionário para armazenar as posições anteriores dos veículos
    previous_positions = {}

    # Variáveis para contar ultrapassagens
    ultrapassagens_direita = 0
    ultrapassagens_esquerda = 0

    # Variável para medir o tempo de processamento
    total_detection_time = 0
    detection_count = 0

    # Variáveis para cálculos de precisão, recall, F1-score
    false_positive_count = 0
    true_positive_count = 0
    false_negative_count = 0
    total_vehicle_detections = 0

    # Coordenada da linha horizontal
    horizontal_line_y = int(frame_height * 0.2)

    # Coordenadas das linhas oblíquas ajustadas para alinhar com as linhas da estrada
    left_line_start = (int(frame_width * 0.3), frame_height)
    left_line_end = (int(frame_width * 0.5), horizontal_line_y)
    right_line_start = (int(frame_width * 0.7), frame_height)
    right_line_end = (int(frame_width * 0.5), horizontal_line_y)

    # Função para verificar se um ponto está à esquerda ou à direita de uma linha
    def is_left_of_line(pt, line_start, line_end):
        return (line_end[0] - line_start[0]) * (pt[1] - line_start[1]) - (line_end[1] - line_start[1]) * (pt[0] - line_start[0]) > 0

    # Função para verificar se um ponto está na área central entre as linhas oblíquas
    def is_in_central_area(pt):
        return is_left_of_line(pt, left_line_start, left_line_end) and not is_left_of_line(pt, right_line_start, right_line_end)

    # Loop para ler e processar cada frame do vídeo
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Verificar se é hora de processar o frame atual
        if frame_count % frame_interval == 0:
            # Aplicar Canny Edge Detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 50, 150)

            start_time = time.time()
            # Fazer a detecção e rastreamento
            results = model.track(frame, persist=True, stream=True)
            detection_time = time.time() - start_time
            total_detection_time += detection_time
            detection_count += 1

            # Iterar sobre as detecções e desenhar as caixas delimitadoras
            current_positions = {}
            for result in results:
                for box in result.boxes:
                    if is_vehicle(int(box.cls[0])):  # Verificar se a classe é um veículo
                        x1, y1, x2, y2 = box.xyxy[0].tolist()  # Convertendo para lista de inteiros
                        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                        vehicle_id = box.id[0].item() if box.id is not None else None  # Obter o ID do veículo

                        if vehicle_id is not None:
                            current_positions[vehicle_id] = (x1, y1, x2, y2)

                            # Verificar se temos a posição anterior deste veículo
                            if vehicle_id in previous_positions:
                                prev_x1, prev_y1, prev_x2, prev_y2 = previous_positions[vehicle_id]

                                # Ponto atual e anterior do veículo
                                current_point = ((x1 + x2) // 2, (y1 + y2) // 2)
                                previous_point = ((prev_x1 + prev_x2) // 2, (prev_y1 + prev_y2) // 2)

                                # Verificar se o veículo está nas ROIs esquerda ou direita
                                is_in_left_roi = is_left_of_line(current_point, left_line_start, left_line_end) and current_point[1] > horizontal_line_y
                                is_in_right_roi = not is_left_of_line(current_point, right_line_start, right_line_end) and current_point[1] > horizontal_line_y
                                is_in_central_roi = is_in_central_area(current_point) and current_point[1] > horizontal_line_y

                                if (is_in_left_roi or is_in_right_roi) and not is_in_central_roi:
                                    # Calcular o deslocamento horizontal
                                    displacement = current_point[0] - previous_point[0]

                                    # Determinar se o veículo está ultrapassando pela esquerda ou direita (invertido para câmera traseira)
                                    if displacement > 0 and is_in_left_roi:
                                        direction = "Esquerda"
                                        ultrapassagens_direita += 1
                                        # Adicionar texto indicando a direção da ultrapassagem
                                        cv2.putText(frame, f'Ultrapassando pela {direction}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                                    elif displacement < 0 and is_in_right_roi:
                                        direction = "Direita"
                                        ultrapassagens_esquerda += 1
                                        # Adicionar texto indicando a direção da ultrapassagem
                                        cv2.putText(frame, f'Ultrapassando pela {direction}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                            # Atualizar a posição anterior do veículo
                            previous_positions[vehicle_id] = (x1, y1, x2, y2)

                            # Desenhar a caixa delimitadora
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                            true_positive_count += 1
                            total_vehicle_detections += 1
                        else:
                            # Contar falsos positivos
                            false_positive_count += 1

            # Calcular falsos negativos: Veículos que não foram detectados neste frame
            detected_ids = set(current_positions.keys())
            all_ids = set(previous_positions.keys())
            false_negative_count += len(all_ids - detected_ids)

            # Desenhar as linhas das ROIs na imagem
            cv2.line(frame, left_line_start, left_line_end, (255, 0, 0), 2)
            cv2.line(frame, right_line_start, right_line_end, (255, 0, 0), 2)
            cv2.line(frame, (0, horizontal_line_y), (frame_width, horizontal_line_y), (255, 0, 0), 2)  # Linha horizontal

            # Escrever o frame processado no arquivo de saída
            out.write(frame)

        # Adicionar um delay para simular visualização em tempo real (ajuste conforme necessário)
        cv2.waitKey(int(1000 / fps))

    # Liberar o vídeo
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Calcular a porcentagem de falsos positivos e falsos negativos
    false_positive_percentage = (false_positive_count / (false_positive_count + total_vehicle_detections)) * 100 if (false_positive_count + total_vehicle_detections) > 0 else 0
    false_negative_percentage = (false_negative_count / total_vehicle_detections) * 100 if (false_negative_count + total_vehicle_detections) > 0 else 0

    # Calcular precisão, recall e F1-score
    precision = (true_positive_count / (true_positive_count + false_positive_count)) * 100 if (true_positive_count + false_positive_count) > 0 else 0
    recall = (true_positive_count / total_vehicle_detections) * 100 if total_vehicle_detections > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Armazenar os resultados
    average_detection_time_ms = (total_detection_time / detection_count) * 1000 if detection_count > 0 else 0
    results_list.append({
        'Modelo': model_file,
        'Tempo Médio de Detecção (ms)': average_detection_time_ms,
        'Total de Detecções': total_vehicle_detections,
        'Falsos Positivos': false_positive_count,
        'Taxa de Falsos Positivos (%)': false_positive_percentage,
        'Falsos Negativos': false_negative_count,
        'Taxa de Falsos Negativos (%)': false_negative_percentage,
        'Precisão (%)': precision,
        'Recall (%)': recall,
        'F1-Score (%)': f1_score
    })

# Criar um DataFrame com os resultados
df_results = pd.DataFrame(results_list)

# Salvar o DataFrame em um arquivo Excel
excel_path = 'Comparacao_Modelos_YOLO.xlsx'
df_results.to_excel(excel_path, index=False)

# Fazer download do arquivo Excel
files.download(excel_path)

# Fazer download do vídeo processado
for model_file in model_files:
    files.download(f'output_video_{model_file}.mp4')


Saving Uniden - Dash View 30R Dash Cam Rear Camera Footage (Day) - Hero.mp4 to Uniden - Dash View 30R Dash Cam Rear Camera Footage (Day) - Hero.mp4
Arquivo Uniden - Dash View 30R Dash Cam Rear Camera Footage (Day) - Hero.mp4 carregado com sucesso.
Analisando com o modelo yolov8m.pt...


100%|██████████| 49.7M/49.7M [00:00<00:00, 257MB/s]


A saída de streaming foi truncada nas últimas 5000 linhas.
0: 384x640 3 cars, 819.8ms
Speed: 2.1ms preprocess, 819.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 812.5ms
Speed: 2.5ms preprocess, 812.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 833.0ms
Speed: 2.1ms preprocess, 833.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 815.3ms
Speed: 2.4ms preprocess, 815.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 810.0ms
Speed: 2.1ms preprocess, 810.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 825.0ms
Speed: 1.9ms preprocess, 825.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 817.5ms
Speed: 1.8ms preprocess, 817.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 806.0ms
Speed

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>